# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [12]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [13]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [15]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, 1000000);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print('到達頁面底端...')
        break

    last_height = new_height
    print('網頁更新中...')
    
# From https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端...


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [18]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [19]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

----------------------------------------------------------------------

[0] 2020上半年經濟前景渾沌　外銀點出5大投資趨勢主題

▲2020上半年全球經濟前景尚未明朗化，外銀點出5大投資主題，可望有強韌表現。（圖／pixabay）
記者紀佳妘／台北報導
今（2020）年全球經濟前景充滿不確定性，打亂投資人布局方向。滙豐私人銀行對此提出5大投資主題，預料這些主題將在上半年全球情勢尚未明朗的情況下，依然具有保持強韌的表現。
首先是亞洲消費動力，滙豐私人銀行北亞首席市場策略師何偉華指出，亞洲的富裕中產階級消費者數量龐大，可帶來具吸引力的結構性成長機遇。
何偉華表示，亞洲私人財富顯著累積、大陸空巢家庭數目上升，以及成長迅速的東協年輕城市人口，都是亞洲消費穩健增長的主要動力。雖然面對全球不明朗因素，但可在亞洲個人服務、高檔消費品、電子商務、娛樂、旅遊、教育、醫療保健和金融服務領域物色優質增長機會。
第二是重塑新大陸，把握大陸經濟轉型、科技升級、金融產業開放和刺激經濟政策帶來的結構增長機會。何偉華表示，大陸的半導體淨進口量龐大，總值2,270億美元占2018年進口總額21%，這是大陸積極尋求科技自主和升級的原因之一。
何偉華認為，科技業是大陸最大的單一進口產業，而台灣的半導體公司將受惠大陸對晶片急速成長的需求，其中包括明年第五代流動通訊（5G）的面世、更趨輕薄的智能手機和無人駕駛技術，都帶動對高端晶片和電子零組件需求的強勁成長，同時也看好受惠於大陸結構性改革和市場開放的公司，並在大陸金融股和偏向內需市場的股票尋求個別機遇，包括粵港澳大灣區優惠政策的股票等。
第三是亞洲企業債券機遇和新興市場債券以息差為重點，強勁的當地基本面和結構增長潛力，加上央行寬鬆政策，都對亞洲及新興市場美元債券和本地貨幣債券有利，相比已開發市場的債券，前者提供更具吸引力的追尋息差機會。
從資金流的技術分析來看，全球負收益債市總值高達11.7兆美元，各地追求收益的投資者將轉投較高收益的資產，因此亞洲和新興市場企業債券很可能吸引追求高收益的資金流入。
第四為亞洲高股息增長策略，何偉華指出，回顧過去十年，亞洲高息股的總回報勝於MSCI亞洲（日本除外）指數，預期具優質增長潛力的亞洲企業將更有可能維持未來配息，甚至調高配息率，讓高息成長股受

加權指數於漲多後隨國際股市拉回修正，雖然存在不少變數，不過低利率環境仍有利延續長多格局，可留意高毛利率且具穩定至成長獲利能力個股，不僅營運爆發力強，也較容易受到市場青睞。
文／黃俊超
二○二○年第一個交易日，加權指數漲逾百點，然而在一月三日，美軍轟炸伊拉克巴格達機場，造成伊朗革命衛隊指揮官蘇雷曼尼喪生，後伊朗還擊美軍駐伊軍事基地，中東局勢再次升溫，油價、金價同步攀高，其中金價漲至六年多來新高，股市則走跌。一九年一月三日，為台股全年低點，而二○年一月三日，則是波段高點。
元月變數較多
美國道瓊、S&P 500與Nasdaq指數相對有撐，維持在短期均線附近，跌勢較重的是費城半導體指數，一度修正至月線附近。連動到國際股市，中國上證未受影響，續沿五日線彈升，香港恆生也在跌破十日線後趨穩，韓國與法國月線具支撐力道，日本與德國則回到季線附近，整體而言都還仍是多頭，屬短線上的震盪。
在台股方面，加權指數四個交易日最大下跌四二○點、跌幅三．四四％，並跌破月線，櫃檯指數相對較重為四．五一％，最低至季線附近。波段性的漲勢過後出現拉回，屬於正常能量釋放與籌碼調整，何況一月十一日將舉辦總統大選，一月十五日美中簽署貿易協定，一月二十日就將封關進入農曆年節假期。
加權指數創高後連續四個交易日下跌，下跌時平均日成交量能為一六三六億元，略高於之前上漲時平均量能，外資共賣超二○九億元，日ＫＤ指標已降至五○中值之下，ＭＡＣＤ轉為負值，布林通道收斂，Put/Call Ratio降到一○○％以下，短線轉震盪整理，可暫時避開，待量能萎縮、籌碼沉澱，整理過後則利多方再起。
今年元月交易天數將少於往年，但是變數卻不少，總統大選結果關乎到正在對峙的全球兩大強權，美國與中國對於台灣未來的態度，也可能會成為美中貿易戰當中的關鍵因素之一，在藍綠兩黨明顯已經選邊站的狀況下，誰能出線將會影響台灣未來發展方向。政治歸政治、經濟歸經濟，基本上不會發生在現實生活當中。
美中貿易戰火下，台灣是聯合國認證的受惠者，經濟成長率勝過韓國、新加坡與反送中抗爭下的香港，而台商回流影響層面更為廣泛，除了投資帶動工業用地需求與工作機會外，對股市也帶來正向加分效果。台灣地小人稠，也沒有太多資源，然而最美的風景是人，最強的武器是經濟底蘊與科技實力！
熱錢party尚未散場
除了外部的不可抗力之外，盤面上接下來必須留意的是去年十二月營收，等同於全年營

記者周康玉／台北報導
選前之夜，電子五哥緯創（3231）今（10）日舉行2019年終尾牙晚會，席開780桌，光是參與員工人數逾7800人。緯創董事長林憲銘每年都會親自抽出現金獎給幸運同仁，據了解，今日現金獎不會低於去年的60萬元，總金額至少破千萬。林憲銘上台致詞除了宣布今天獎金比去年好之外，還在結語時提醒同仁，明天一定要去投票，非常應景。
緯創尾牙於台北市南港展覽館舉辦，晚間6點登場，開場前一分鐘，還有倒數影片，並且在台上播放今年成果，由於聲光音效極為強烈，並且逐一回顧公司各部門去年的工作成果，被點到的部門員工歡呼連連，也成為晚會的熱烈開場。
緯創今年尾牙主題是「2020迎風而起 逆風高飛」，開場影片除了透露緯創去年在印度和越南等海外擴廠進度外，也特別透露，去年在筆電業務通過了五大品牌廠審查，今年產能會有翻倍成長。
林憲銘上台致詞表示，去（2019）年一開始就意識到是非常不平靜的一年，從總體經濟來看，加上區域政治衝突 ，貿易戰等， 讓去年全球經濟並沒有非常的順暢，因此他在去年初對於全年營運做了保守的估計。
林憲銘表示，一年來下來，經過同仁的努力，雖然在總體經濟不是很好，全年營收並沒有成長，但緯創的體質改變，去年第三季獲利就比去年同期有相當幅度的成長。 林憲銘表示，雖然他不能做預測，但他相信2019年的獲利會比前（2018）年成長。
至於員工最關心的現金獎，林憲銘也掛保證「今晚的抽獎比去年要多的，大獎比去年多；就連剛剛才核發的年終獎金，也比去年要多！」讓台下員工鼓掌叫好。
林憲銘也難得在公開場合談及員工調薪，他表示，除了每年四月份正常調薪以外，最近又推動了員工提撥公司持股的辦法，就是員工加薪3%，員工加薪同時可儲蓄，公司把營運成長跟同仁共享是領導層的責任。
緯創在去年獲得MSCI的雙A企業，林憲銘表示，包括台積電等一共才五個公司獲得此評選，是因為緯創在環保公益公司治理等各方面多年的努力，去年終於被看到，這些都是員工努力的成果。
林憲銘說，尾牙就是和員工說感謝的日子，並且特別在致詞結束前，提醒台下同仁「明天一定要去投票！」在選前之夜，特別點題。
----------------------------------------------------------------------

[21] 主計總處公布11月經常性薪資42,130元　年增率為2.16%　

▲ 行

▲華銀全台發送5萬個發財金。（圖／華南銀行提供）
記者吳靜君／台北報導
農曆新年將近，華南銀行為歡喜迎鼠年，今年特別在農曆除夕前3天，從1月20日至22日送發財金。客戶只要在這3天親自至華南銀行各分行理財櫃台即可領取，每天限量88個，讓華南銀行客戶在鼠年鼠(數)錢發發發。
華南銀行每年所準備生肖系列之「發財金」皆非常搶手，有很多客戶正收集未來12生肖一套開運「發財金」。大家都知道「好的開始是成功的一半」，對於2020鼠年而言，更代表著華南銀行帶著滿滿的祝福要與客戶迎接新的一年。.
今年5萬個華銀鼠年開運發財金，設計成Q版可愛立體造型，萌感吉祥物雙手捧著閃閃發亮的發財金，擺在家裡、辦公室或是店家的櫃台，都相當討喜、吉利。
華南銀行的限量發財金，每年都特地選在良辰吉時，擺放在銀行的金庫裡吸收滿滿的財氣，要讓理財客戶在鼠年都可以鼠(數)錢發大財。
----------------------------------------------------------------------

[34] 台塑四寶去年獲利1270億元、縮水4成　南亞衰退最多

▲台塑四寶業績說會，資料照。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台塑四寶今（10）日舉行12月業績說會，並公布去年2019年財報成績單，合計四大公司（台塑、南亞、台化、台塑化），受美中貿易戰與國際油價波動衝擊，合計稅後純益1,271億元，比起前年的2,109億元，大幅縮水四成，其中台塑獲利年減26.1%，衰退幅度為四寶中最少；每股純益為5.86元，拿下台塑四寶「獲利王」頭銜。
台塑四寶去年合併營收1.46兆元，較前年1.73兆元減少15.6%。台塑指出，因液鹼、AN、MMA、ECH、AE及NBA等產品售價下跌，利差縮小，影響獲利表現。
而南亞表示，主要是各產品市場供給大多增加或持平，而因競爭激烈、產品價格偏低，部份產線不敷成本而停車減產，導致營收減少。其中以化工產品(EG及BPA為主)減少最多，其次為聚酯產品。全年稅後純益230.69億元，年減56.2%，衰退幅度是四寶最大，每股純益2.91元。
台化表示，主要ARO-3異常，後段PX製程僅開一線，PX減少外售及萃餘油等回售塑化減少共影響34億元；SM自用增加，減少外售4.5億元。導致獲利減少，去年稅後利益為299.91億元，年減37.9%，每股稅後盈餘5.14元。

▲為確保選舉期間供電穩定，台電公司在全台24個營業區處動員近二千名人力，是平時輪值人力的兩倍。（圖／台電提供）
記者林淑慧／台北報導
澎湖本島唯一一座火力發電廠今（10）日早上傳出機組跳電，導致全澎湖大停電，台電公司回應表示，澎湖尖山電廠上午10時49分因機組故障，造成饋線跳脱，共26381戶停電。經電廠緊急處理後，於11時19分全部復電。
2020總統大選明（11）日登場，為確保選舉期間供電穩定，台電公司成立「穩定供電應變小組」，由總經理鍾炳利親自坐陣督軍，在全台24個營業區處動員近二千名人力佈署，確保投開票作業順利進行。
台電總管理處已成立「第15任總統副總統暨第10屆立法委員選舉穩定供電應變小組」，為因應此次大選，各區處輪值巡修調度人力達916名，另額外加派1,054名支援搶修人力，總計動員1,970人，是平時輪值人力的兩倍。
另外，台電表示，因應中央選舉委員會及地方政府需求，也將在各選務機關設置共22具大型發電機協助應變，另備妥127具小型發電機隨時待命，力求供電穩定使選務工作順利進行。
----------------------------------------------------------------------

[48] 台股萬點到年線有強勁支撐　五大布局方向要抓穩

▲ 台股在萬點~年線區間仍有支撐。（圖／記者湯興漢攝）
記者李瑞瑾／綜合報導
台股在中美將簽訂第一階段貿易協議激勵下，持續高檔震盪，展望後市，台新2000高科技基金經理人沈建宏表示，據歷史經驗，1~2月為作夢行情及個股表現格局，預期加權指數在萬點~年線附近，仍有強勁支撐，第一季指數仍可樂觀期待，建議可利用短線的急跌分批逢低布局。題材部分則看好成長性強勁的5G、半導體族群，另外，傳產股跟隨選前政策作多，亦具個別次族群行情，可留意相關政策受惠股。
沈建宏指出，分析目前國際環境有五大關注焦點：一、中美貿易戰將達成第一階段協議，但華為禁運令短期可能不會鬆綁，中美貿易戰對股市衝擊性降低；二、美伊衝突升溫，提供股市技術面漲多拉回理由，但不致扭轉股市方向。
第三，ECB將重啟QE，FED也恢復擴表，有助資金行情底部承接；四、中國製造業PMI指數止跌，美國製造業ISM指數第一季預估也可望止跌回溫；五、美國眾議院通過對川普彈劾的程序，但參議院為共和黨掌控,成案機率低。整體而言，國際系統風險不高

▲台塑董事長林健男。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台塑企業今（9）日舉辦2019年暮年會，也就是尾牙，台塑董事長林健男表示，今年福欣新建冷軋廠將完工投產，為營運增添強勁新動能，並將投資 58 億美元進行全球佈局，加速推動未來5年內完工的投資案。
林健男表示，2019年是充滿變數的一年，受到中美貿易戰影響，全球經濟景氣走緩，導致石化業景氣低迷，本公司及轉投資公司獲利都衰退，但在技術研發及AI應用上的努力有收獲。
林健男更提到，雖然中美1月15日即將簽署貿易談判第一階段協議，不過，影響經濟前景的不確定因素還有很多，尤其未來三年中國大陸及美國石化產品新產能大量開出，但必須更加審慎應對，逆風中求穩，加緊腳步，繼續秉持　總裁的理念，深化AI及研發，強化企業經營體質，以提升市場競爭力。
此外，林健男也宣布，去年在大家努力之下，美國德州新建OL-3及HDPE廠已順利運轉，今年福欣新建冷軋廠，也將完工投產，加上台南沙崙染敏電池試產成功，可為未來營運，增添新的成長動能。
----------------------------------------------------------------------

[63] 影／退休金該如何規劃？　台股達人分享攻略

退休後的資金運用分配非常重要，光靠政府給的退休金，恐怕很難過舒適生活，在《雲端最有錢》節目中，技術分析大師杜金龍和存股教授謝士英就分享他們如何規劃退休金，甚至還靠著股票每個月都有多些加菜金。
杜金龍表示，自己一個月生活開銷雖是7萬元，但政府的勞保、勞退加起來，至少可提供每月2萬多元，剩下的5萬元就是靠演講稿費、小孩孝親費和股票加菜金。
杜金龍退休後，每月會拿出3成資金投入股市， 賺個買菜錢，至於為什麼是三成？因為杜金龍是用30年餘命計算退休金目標，而根據統計，國人退休後平均餘命大約20年，因此，把退休金「高估」的部分，也就是3成左右資金用來進行風險較高的投資。
謝士英則表示，自己就是挑選好公司按部就班的存股，每年也都會重新檢視公司去年的營運績效是否有比前年好，如果有就會考慮加碼，像是最近就加了玉山金（2884）、長興（1717）、佳世達（2352），原因除了獲利表現優異外，還有認為它們未來發展很大的潛能。
----------------------------------------------------

文／洪綾襄
過去1年，信用卡市場戰況激烈。根據金管會統計，2019年1~11月，33家信用卡發卡機構總共新發出了306萬張信用卡，總流通卡數達到4709萬張，飆破雙卡風暴時期的4549萬張。
儘管總流通卡數排名不變，仍是中國信託居冠，國泰世華居次，第3為玉山銀行，第4是台新銀行，第5為台北富邦，但在內涵上，已起了極大變化。
其中，新發卡王由玉山銀行拿下，去年1~11月玉山銀共發出117.51萬張信用卡，不但比第2名的台新銀行多出19.46萬張，僅略遜於第3名中國信託加上第4名國泰世華的總發卡數 （149.22萬張）。
而百萬信用卡俱樂部也有新成員加入，國泰世華Costco聯名卡以210萬張蟬聯冠軍，其次為中國信託LINE Pay聯名卡180萬張，再來為玉山家樂福卡150萬張，台新新光三越卡120萬張，最新進榜的則是奠基於HappyGo千萬會員的遠銀快樂信用卡。
不過中國信託也未因此鬆懈，元旦前宣布慶祝LINE Pay卡的發卡量（含信用卡與一卡通聯名卡）突破300萬張，權益再升級，與更多通路店家合作，祭出消費最高享22% LINE點數等的回饋繼續搶市。
科技助力　刷卡回饋更多元
現金卡與信用卡造成的「雙卡風暴」至今14年，去年信用卡市場才再度熱絡起來，關鍵在於近期中型銀行重兵部署數位金融，才能搶灘占市、精準行銷，不再讓發卡大行專美於前。玉山和台新就是個中代表。
再加上，6月還有3家純網銀即將開業，自然得先把市場搶起來。
銀行力推，為什麼消費者也甘願埋單？主因在於去年推出的新卡權益讓消費者真的很有感。以堪稱2019年年度神卡的玉山Ubear信用卡為例，其主打的都會年輕白領習性即為「每天去超商買咖啡、喜歡網路購物、周末會和朋友看電影」，該卡就設計出「一般消費1%、網購5%、超商8%、電影20%現金回饋」的權益，完全貼合目標受眾的需求，因而大受歡迎。
尤其，玉山Ubear主要訴求年輕族群，被視為未來網銀的主力市場，玉山透過發行信用卡提前「圈粉」，被視為網銀市場開放競爭的前哨戰。
更何況，數位化後，銀行策略工具也愈多元。「隨著消費者對虛擬幣、電子支付、數位帳戶等新金融工具接受度愈來愈高，銀行發卡的策略與內涵已和10年前完全不同了。」
延伸閱讀：
常勝軍！6度贏得最佳金控CSR金質獎》玉山金把服務做到手機裡 12分鐘快速核卡
純網銀3條鯰魚入海 會不會變鯊魚？
--------

▲華碩筆電。（圖／記者湯興漢攝）
記者姚惠茹／台北報導
華碩（2357）今（9）日公布去年12月集團營收為327億元，年增6%；品牌營收為288億元，年增4%；合計去年第4季集團營收為973億元，年增7%；去年第4季品牌營收為874億元，年增6%；2019年全年集團合併營收為3513.16億元，年減0.8%。
華碩表示，自2019年的下半年起，華碩品牌營收相較前年同期已持續6個月正成長，第4季品牌營收年成長6%，板卡、筆記型電腦、電競產品的出貨量均達正成長，歐洲及亞太地區的成長強勁，營運趨勢持續朝正向發展。
華碩指出，2019年第4季處理器的供應狀況或有延遲，區域性市場的營運調整壓力增加，並因策略性增加對行銷及銷售通路的投資布局，去年第4季的獲利率將因此承受相當壓力。
展望2020年，華碩表示，公司對2020年產業持審慎樂觀看法，將展現積極的企圖心及執行力，設定具挑戰性的成長目標。
----------------------------------------------------------------------

[81] 認定託付寶踩紅線！金管會：「不能保證獲利是底線」　街口承諾下架廣告

▲街口支付執行長胡亦嘉。（圖／記者湯興漢攝）
記者吳靜君／台北報導
街口金融科技公司「託付寶」廣告，打出保證收益1.5%，金管會證期局今（9）日強調，投信投顧法規有一項「基本原則」就是不能夠「保證獲利」即使申請監理沙盒，也不能夠挑戰這個底線。而街口支付在各辦公大樓、捷運站上，大張旗鼓張貼「託付寶」廣告，金管會表示，業者已經承諾會下架廣告。
金管會證期局副局長張振山表示，根據投信投顧法規以及證券投資信託管理規則第19條與第22條規定，最基本的原則就是不可以「保證獲利」。
業者想要透過監理沙盒的方式來採取試驗，證期局也表示，沙盒是如果對於法規定的不明確、有模糊或者灰色的地帶，業者有新的創意想要試驗，就可以來申請，但有一些基本原則及法規，就是要合法合規。現在街口還未申請監理沙盒試驗，最後還是要看提出的情況來看，但是「保證獲利」是依據實質情況來審核，不論業者是用什麼字眼或名義。
根據街口的規劃，如果超過1.5%的獲利則是歸街口所用，證期局也表示，這部分也已經違反證券投資信託事業管理規則第19條規定，目前也沒有任何基金有這樣做。
------------------------

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（9）日開高走高，開盤不到1分鐘即飆漲逾百點、站回11,900點關卡，最高甚至一度逼近1萬2千點，終場收在11,970點、上漲153點或1.3%，成交量1,322.76億元。
電子三雄中，台積電（2330）以337.5元作收、上漲2.43%；鴻海（2317）收在87.1元、上漲0.69%；股王大立光（3008）以4865元作收、上漲1.67%。
觀察類股表現，油電燃氣類股跌逾1%，其他類股全數收紅，電機股漲逾3%、半導體漲2.33%，觀光、電子、電子零組件、光電等漲幅在1%以上。
電機類股中，亞德客-KY（1590）盤中漲停，上銀（2049）收漲6.06%、中興電（1513）、力山（1515）、永冠-KY（1589）漲幅逾4%；正峰新（1538）、正道（1506）、和大（1536）相對疲弱，下跌超過1%。
除了台股在今日反彈走高之外，其他亞洲市場如日股、韓股、港股、陸股也都走揚。其中日經指數上漲逾2%，南韓綜合指數、恆生指數上漲1%以上。
----------------------------------------------------------------------

[93] 元大15年EM主權債ETF掛牌　掌握新興債市

▲元大15年EM主權債ETF小檔案。（資料來源：元大投信整理，2020/1）
記者周康玉／台北報導
元大投信今（9）日於櫃買中心為元大15年EM主權債ETF（證券代碼 00870B）（本基金有一定比重投資於非投資等級之高風險債券）舉行掛牌典禮，掌握新興市場投資機會。
元大15年EM主權債ETF追蹤ICE新興市場 15 年期以上美元主權債券指數，指數殖利率約4%，投資優勢包含三大項。第一，以美元計價主權債券，布局新興市場不用承擔新興貨幣匯率風險。第二，投資國家分散，指數成分股目前主要包含阿烏地阿拉伯、印尼等26個國家，可分散單一國家風險。第三，息值有助價格長期往上，新興市場債相對高的息值讓指數長期呈現走揚。
根據國際貨幣基金組織（IMF）最新的《世界經濟展望》中看好新興市場整體的經濟成長率將回升到4.6%，在全球中相對突出。
再從資金面來看，自2019年聯準會啟動降息以來，各國寬鬆財政與貨幣政策陸續發酵，多家國際投資機構認為美國聯準會貨幣政策持續偏鴿，有利整體債市表現，而市場